# CODIGO DE RE-ENTRENAMIENTO SOFT

In [ ]:
#Posibles paquetes necesarios para el desarrollo del proyecto. No harán falta todos siempre. Seleccionar los necesarios
import os
import numpy as np
import pandas as pd
import pickle
import cloudpickle #Mismas funciones de pickle de guardar modelos + funciones de guardado de las funciones específicasimport seaborn as sns
import sqlalchemy as sa
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline
%config IPCompleter.greedy = True

#Formato sin notación científica
pd.options.display.float_format = '{:15.2f}'.format 

#Automcompletar rápido
%config IPCompleter.greedy=True

#Desactivar los warnings
import warnings
warnings.filterwarnings("ignore")

#Representación visual de un pipeline
from sklearn import set_config
set_config(display = 'diagram') #diagram/text 

#Transformación de variables
from janitor import clean_names
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

#Modelos ML: CLASIFICACIÓN
from xgboost import XGBClassifier
#Métrica de error: Habrá que incluir la que necesitemos
from sklearn.metrics import roc_auc_score

# PRECIO_M2 IDEALISTA
#Ruta del proyecto
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB'

#Nombre del fichero de datos
nombre_fichero_datos = 'precios_idealista_Dic24.csv'

#Cargar los datos
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos
precio_m2 = pd.read_csv(ruta_completa)

def modificaciones_idealista(precio_m2):
    precio_m2 = precio_m2.loc[1:,['table__cell 2','icon-elbow']] \
    .rename(columns = {'table__cell 2':'precio_m2','icon-elbow':'neighbourhood_group'})
    
    precio_m2['precio_m2'] = precio_m2['precio_m2'].str.replace(' €/m2','').str.replace('.','').astype('int')
    
    return(precio_m2)
precio_m2 = modificaciones_idealista(precio_m2)
precio_m2


# LISTINGS AIRBNB
#Ruta del proyecto
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB'

con = sa.create_engine('sqlite:///C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB/02_Datos/01_Originales/airbnb2025.db')

#Nombre del fichero de datos
nombre_fichero_datos = 'airbnb2025.db'

#Cargar los datos
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos
listings = pd.read_sql('listings', con).drop(columns='index')#.set_index('id')



# LISTINGS_DET AIRBNB
#Ruta del proyecto
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB'

con = sa.create_engine('sqlite:///C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB/02_Datos/01_Originales/airbnb2025.db')

#Nombre del fichero de datos
nombre_fichero_datos = 'airbnb2025.db'

#Cargar los datos
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos
listings_det = pd.read_sql('listings_det', con).drop(columns='index')#.set_index('id')

pd.set_option('display.max_columns', None)
def preparacion_df(listings_det, listings, precio_m2):
    
    a_eliminar = [
     'listing_url',
     'scrape_id',
     'last_scraped',
     'source',
     'description',
     'neighborhood_overview',
     'picture_url',
     'host_url',
     'host_name',
     'host_location',
     'host_about',
     'host_picture_url',
     'host_neighbourhood',
     'neighbourhood',
     'neighbourhood_group_cleansed',
     'minimum_minimum_nights',
     'maximum_minimum_nights',
     'minimum_maximum_nights',
     'maximum_maximum_nights',
     'minimum_nights_avg_ntm',
     'maximum_nights_avg_ntm',
     'calendar_updated',
     'calendar_last_scraped',
     'host_thumbnail_url']
    listings_det = listings_det.drop(columns = a_eliminar)    
    
    # 1. Normalización del nombre de barrios
    def join_listings(listings_det, listings):
        listings['neighbourhood_group'] = listings['neighbourhood_group'].replace({
            'Fuencarral - El Pardo': 'Fuencarral',
            'Moncloa - Aravaca': 'Moncloa',
            'San Blas - Canillejas': 'San Blas'
        })
        listings_det = pd.merge(left=listings_det, right=listings[['id', 'neighbourhood_group']], how='left', on='id')
        return listings_det

    listings_det = join_listings(listings_det, listings)

    # 2. Join con precios por metro cuadrado
    def join_df(listings_det, precio_m2):
        df = pd.merge(left=listings_det, right=precio_m2, how='left', on='neighbourhood_group')
        return df
    df = join_df(listings_det, precio_m2)
    return df

df = preparacion_df(listings_det, listings, precio_m2)


#Traemos las variables que corresponden. Todas las funciones que usamos anteriormente en los notebooks están aquí 
pd.set_option('display.max_columns', None)

def calidad_datos(df):
    
    # Limpiamos los nombres de las variables por si hubiera algun carácter extraño
    df = clean_names(df)
    
    # Borramos duplicados
    #df.drop_duplicates(inplace = True)
    
    # Creamos una tabla temporal con una copia de df para trabajar sobre ella
    temp = df.copy()
    
    #PRICE
    def price(temp):
        
        # Asegurarse de que 'price' es numérico
        temp['price'] = temp['price'].replace(r'[\$,]', '', regex=True).astype(float)

        # 1. Promedio por 'neighbourhood_cleansed' y 'accommodates'
        grouped_price = temp.groupby(['neighbourhood_cleansed', 'accommodates'])['price'].mean()

        # 2. Promedio por 'accommodates'
        grouped_accommodates_price = temp.groupby('accommodates')['price'].mean()

        # 3. Promedio general
        global_price_mean = temp['price'].mean()

        # Función de imputación
        def imputar_precio(registro):
            if pd.isna(registro['price']):
                val = grouped_price.get((registro['neighbourhood_cleansed'], registro['accommodates']), np.nan)
                if pd.isna(val):
                    val = grouped_accommodates_price.get(registro['accommodates'], np.nan)
                if pd.isna(val):
                    val = global_price_mean
                return round(val) if pd.notna(val) else np.nan
            else:
                return registro['price']

        # Aplicar la imputación
        temp['price'] = temp.apply(imputar_precio, axis=1)   
        return temp        
    temp = price(temp)
    
    # Seleccionar solamente los inmuebles con un precio superior a 20€ por noche
    temp = temp.loc[temp['price']>=20]
    
    #BEDS
    def imputar_beds(registro):
        if pd.notna(registro['beds']):
            return registro['beds']
        if registro['accommodates'] <= 2:
            return 1
        elif registro['accommodates'] <= 4:
            return 2
        elif registro['accommodates'] <= 6:
            return 3
        else:
            return 4
        
    temp['beds'] = temp.apply(imputar_beds, axis=1).astype(float) 
    
    #BEDROOMS
    def imputar_bedrooms(registro):
        if pd.notna(registro['bedrooms']):
            return registro['bedrooms']
        if registro['beds'] <= 2:
            return 1
        elif registro['beds'] <= 4:
            return 2
        elif registro['beds'] <= 6:
            return 3
        else:
            return 4
        
    temp['bedrooms'] = temp.apply(imputar_bedrooms, axis=1).astype(float) 
    
    #BATHROOMS
    
    #1. bathrooms_text None --> 0
    #2. bathrooms_text quitamos el texto detras del numero
    #3. bathrooms_text sustituimos los NaN generados por 0.5
    #4. bathrooms sustituimos los NaN por los valores de bathrooms_text del mismo registro
    #5. Eliminamos bathrooms_text porque ya tenemos completa la variable bathrooms
      
    def bathrooms(temp):
        temp['bathrooms_text'] = temp['bathrooms_text']\
            .replace(np.nan, '0')\
            .replace(r'[^\d.]', '', regex=True)\
            .replace('', '0.5').astype('float64')
        temp['bathrooms'] = np.where(temp.bathrooms.isna(), temp.bathrooms_text, temp.bathrooms)
        #temp.drop(columns='bathrooms_text', inplace=True)
        return temp
    temp = bathrooms(temp)      
    
    #REVIEWS
    var_reviews = ['review_scores_rating',
                     'review_scores_accuracy',
                     'review_scores_cleanliness',
                     'review_scores_checkin',
                     'review_scores_communication',
                     'review_scores_location',
                     'review_scores_value']
    valor = 0
    def reviews(temp):
        for col in var_reviews:
            temp[col] = np.where(
                temp[col].isna(), 
                temp['review_scores_rating'], 
                temp[col])
        temp[var_reviews] = temp[var_reviews].fillna(valor)
        return temp
    temp = reviews(temp)
    
    #HOST RESPONSE TIME    
    valor = 'no response'
    temp['host_response_time'] = temp['host_response_time'].fillna(valor)
    
    #HOST RESPONSE RATE
    temp['host_response_rate'] = temp['host_response_rate'].replace('%','', regex=True)
    temp['host_response_rate'] = np.where(temp['host_response_rate'].isna(), 0,\
                                          temp['host_response_rate']).astype('float')
    
    #LICENSE
    temp['license'] = np.where(temp['license'].notna(), 'yes', 'no')    
    
    #HAS AVAILABILITY
    temp['has_availability'] = np.where(temp['has_availability'].notna(), 'yes', 'no')

    #HOST IS SUPERHOST
    def host_is_superhost(temp):
        # Función para imputar
        def imputar_is_superhost(registro):
            if pd.isna(registro['host_is_superhost']):
                scores = [
                    registro['review_scores_cleanliness'],
                    registro['review_scores_accuracy'],
                    registro['review_scores_rating'],
                    registro['review_scores_communication'],
                    registro['review_scores_checkin'],
                    registro['review_scores_location'],
                    registro['review_scores_value']
                ]
                # Verificar que todos los puntajes no son nulos y mayores a 4.5
                if all(pd.notna(score) and score > 4.5 for score in scores):
                    return 't'
                else:
                    return 'f'
            return registro['host_is_superhost']

        # Aplicar la imputación
        temp['host_is_superhost'] = temp.apply(imputar_is_superhost, axis=1)

        return temp
    temp = host_is_superhost(temp)
    
    #HOST VERIFICATIONS
    temp['host_verifications'] = np.where(temp['host_verifications']=='[]', "['No']", temp['host_verifications'])
    temp['host_verifications'] = np.where(temp.host_verifications == "['email', 'work_email']", 
                                          "['email']", 
                                          temp.host_verifications)
    
    #MINIMUM NIGHTS / MAXIMUM NIGHTS
    def atipicos_desv_tip(variable, num_desv_tip = 4):
        #sacamos los nulos por ahora
        variable = variable.dropna()
        #calculamos los límites
        media = np.mean(variable)
        sd = np.std(variable)
        umbral = sd * num_desv_tip
        lim_inf = media - umbral
        lim_sup = media + umbral
        #encontramos los índices de los que están fuera de los límites
        indices = [indice for indice,valor in variable.items() if valor < lim_inf or valor > lim_sup]
        return(indices)
    var_atipicos_desv_tip = ['minimum_nights', 'maximum_nights']
 
    # ELIMINAMOS LOS HOTELES PORQUE NO SON NUESTRO OBJETIVO
    temp = temp.loc[~((temp.property_type.str.contains('hotel', case=False, na=False))|
                      (temp.property_type.str.contains('hostel', case=False, na=False))|
                      (temp.room_type.str.contains('hotel', case=False, na=False)))]
    
    #Agrupar categorias raras
    def agrupar_cat_raras(variable, criterio = 0.02):
        #Calcula las frecuencias
        frecuencias = variable.value_counts(normalize=True)
        #Identifica las que están por debajo del criterio
        temp = [cada for cada in frecuencias.loc[frecuencias < criterio].index.values]
        #Las recodifica en 'OTROS'
        temp2 = np.where(variable.isin(temp),'OTROS',variable)
        #Devuelve el resultado
        return(temp2)
    var_agrupar_cat_raras = ['property_type']
    for variable in var_agrupar_cat_raras:
        temp[variable] = agrupar_cat_raras(temp[variable],criterio = 0.02)    
    
    # Eliminamos todas los registros que no tengan 50 variables sin nulos después de todo el proceso de calidad de datos 
    criterio = 50 #Elimina todos los registros que no tengan el número de variables del criterio sin nulos
    temp.dropna(thresh=criterio, inplace=True)
    temp.isna().sum().sort_values(ascending=True)
    
    return temp
df = calidad_datos(df)


pd.set_option('display.max_columns', None)
def gestion_variables(df):
    # 1. Estimación de m2 por número de dormitorios
    condiciones = [df.bedrooms == 0,
                   df.bedrooms == 1,
                   df.bedrooms == 2,
                   df.bedrooms == 3,
                   df.bedrooms == 4,
                   df.bedrooms > 4]
    resultados = [40, 50, 70, 90, 120, 150]
    df['m2'] = np.select(condiciones, resultados, default=-999)

    # 2. Estimación de precio de compra
    df['precio_compra'] = df['m2'] * df['precio_m2'] * 0.8

    # 3. Cálculo de disponibilidad promedio por tipo de habitación
    dispo_private = df[df['room_type'] == 'Private room']['availability_365'].mean() / 365
    dispo_shared = df[df['room_type'] == 'Shared room']['availability_365'].mean() / 365

    # 4. Cálculo del precio total estimado
    def crear_precio_total(registro):
        try:
            price = float(registro.price)
            beds = float(registro.beds)
        except (ValueError, TypeError):
            return np.nan

        room = registro.room_type
        if pd.isna(price) or pd.isna(beds) or pd.isna(room):
            return np.nan

        if beds > 1 and room == 'Private room':
            return price * beds * (1 - dispo_private)
        elif beds > 1 and room == 'Shared room':
            return price * beds * (1 - dispo_shared)
        else:
            return price

    df['precio_total'] = df.apply(crear_precio_total, axis=1).round(2)
    
    # 5. Determinar la ocupación del inmueble
    df['ocupacion'] = ((365 - df.availability_365) / 365 * 100).astype('int')
    
    # 6. Creación de las distancias a un pdi
    from math import radians, cos, sin, asin, sqrt
    def haversine(lat1, lon1, lat2, lon2):

          R = 6372.8 #En km, si usas millas tienes que cambiarlo por 3959.87433

          dLat = radians(lat2 - lat1)
          dLon = radians(lon2 - lon1)
          lat1 = radians(lat1)
          lat2 = radians(lat2)

          a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
          c = 2*asin(sqrt(a))

          return R * c
    #Las coordenadas de la Puerta del Sol serán lat1 y lon1
    lat1 = 40.4167278
    lon1 = -3.7033387

    df['pdi_sol'] = df.apply(lambda registro: haversine(lat1,lon1,registro['latitude'],registro['longitude']),axis=1)

    # 7. Determinar si el piso es rentable
    df['pisos_rentables'] = np.where(df['precio_compra'] / 10 > df['precio_total'] * df['ocupacion'] * 10, 0, 1)
    
    def variables_texto(df):
        if os.path.exists("count_vectorizer.pkl"):
            with open("count_vectorizer.pkl", "rb") as f:
                cv = pickle.load(f)
        else:
            cv = CountVectorizer()
            cv.fit(df["amenities"].astype(str))
            with open("count_vectorizer.pkl", "wb") as f:
                pickle.dump(cv, f)
        #Aplicamos
        caracteristicas = cv.transform(df["amenities"].astype(str))
        
        #Definimos un dataframe con los términos encontrados transformados a variables que mapean las columnas donde se encuentran
        terminos = pd.DataFrame(caracteristicas.toarray(), columns=cv.get_feature_names_out(), index=df.index)
        
        #Concatenar al DataFrame original        
        df = pd.concat([df, terminos], axis=1)    
        return df
    df = variables_texto(df)
    
    return df
df = gestion_variables(df)

id_df = df['id'].copy()


variables_finales = [
'accommodates',
'availability_30',
'availability_365',
'availability_90',
'bedrooms',
'bed',
'calculated_host_listings_count',
'has_availability',
'host_response_rate',
'host_response_time',
'instant_bookable',
'maximum_nights',
'neighbourhood_group',
'price',
'property_type',
'room_type',
'license',
'bathrooms',
'air',
'allowed',
'cleaning',
'hot water iron',
'iron',
'microwave',
'parking',
'refrigerator',
'shampoo',
'tv washer',
'wifi kitchen',
'neighbourhood_cleansed',
'and silverware',
'body',
'clothing',
'clothing storage',
'coffee',
'elevator',
'hair',
'host_identity_verified',
'host_verifications',
'kitchen essentials',
'allowed',
'basics',
'beds',
'dryer bed',
'hair dryer',
'maker',
'microwave hangers',
'oven',
'stove',
'review_scores_rating',
'review_scores_accuracy',
'review_scores_cleanliness',
'review_scores_checkin',
'review_scores_communication',
'review_scores_location',
'review_scores_value',
'host_is_superhost',
'bathrooms_text',
'precio_m2',
'latitude',
'longitude',
'amenities',
'number_of_reviews']


x = df[variables_finales].copy()

target = 'pisos_rentables'
y = df[target].copy()



nombre_pipe = 'pipe_entrenamiento.pickle'
ruta_pipe = ruta_proyecto + '/04_Modelos/' + nombre_pipe
with open(ruta_pipe, mode='rb') as file:
    pipe_entrenamiento = cloudpickle.load(file)

    

pipe_ejecucion = pipe_entrenamiento.fit(x,y)
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'
ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion
with open(ruta_pipe_ejecucion, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion, file)
pipe_ejecucion
